In [9]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [20]:
data = pd.read_excel('C:\Users\user\Documents\GitHub\Deep_Learning_Rainfall\New data.xlsx')
print(data)

FileNotFoundError: [Errno 2] No such file or directory: 'New data.xlsx'

In [16]:


def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

class CNN1_Net(nn.Module):
    def __init__(self,
                 kernel_size_1=5,
        stride_size=1,
        num_channels=10,
        depth_1=1,
        depth_2=5,
        kernel_size_2=5,
        num_hidden=20,
        num_labels=5
                ):
        super(CNN1_Net, self).__init__()
        
        self.classifier=nn.Sequential(
            nn.Conv1d(num_channels, depth_1, kernel_size=kernel_size_1),
            #nn.ReLU(),
            #nn.MaxPool1d(1),
            #nn.Dropout(0.1),
            #nn.Conv1d(in_channels=1, out_channels=1, kernel_size=5)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(depth_2*kernel_size_2, num_hidden),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        self.fc2 = nn.Sequential(
            nn.Linear(num_hidden, num_labels),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
    
    def forward(self,x):
        x=self.classifier(x)
        #x=self.fc1(x)
        #x=self.fc2(x)
        print(x)
        return(x)


    def __init__(self,
                 kernel_size_1=5,
                 stride_size=1,
                 num_channels=1,
                 depth_1=1,
                 depth_2=5,
                 kernel_size_2=5,
                 num_hidden=20,
                 num_labels=16  # Number of columns in your dataset
                ):
        super().__init()  # Use super() without double underscores

        self.classifier = nn.Sequential(
            nn.Conv1d(num_channels, depth_1, kernel_size=kernel_size_1),
        )
        self.fc1 = nn.Sequential(
            nn.Linear(depth_2 * kernel_size_2, num_hidden),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(num_hidden, num_labels),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

    def forward(self, x):
        x = self.classifier(x)
        return x


data = pd.read_excel('New data.xlsx')
# data.rename(columns={'Sales': 'Rainfall'}, inplace=True)  # Rename the 'Sales' column to 'Rainfall'

# X = data.Rainfall.copy()
# y = data.Rainfall.rolling(5).mean().copy()
X = data.iloc[:, 1:].values  # Use all columns except the first one as features
y = data.iloc[:, 1:].values  # Use the same columns as features

batch_size = 5

model = CNN1_Net()
optimizer = optim.Adam(model.parameters(), lr=0.01)  # L2
criterion = nn.CrossEntropyLoss()

model.train()

values = []

for epoch in range(10):
    for batch in iterate_minibatches(X[:len(y)], y, batch_size):
        x_sublist, y_sublist = batch
        X2_tensor = torch.Tensor(np.array(x_sublist)).reshape(1, 1, -1)  # 3 dimensional
        y2_tensor = torch.Tensor(np.array(y_sublist)).reshape(1, 1, -1)  # 3 dimensional
        outputs = model(X2_tensor)
        print(X2_tensor, outputs)

net = nn.Conv1d(1, 1, 5, bias=False)
optimizer = optim.Adam(net.parameters(), lr=0.01)  # L2
running_loss = 0.0

# X = data.Rainfall.copy()
# y = data.Rainfall.rolling(5).mean().copy()
X = data.iloc[:, 1:].values
y = data.iloc[:, 1:].values


X_tensor = torch.Tensor(X).reshape(1, 1, -1)
y = y[4:].to_numpy()
y_tensor = torch.Tensor(y).reshape(1, 1, -1)

ts_tensor = torch.Tensor(X).reshape(1, 1, -1)
kernel = [0.5, 0.5]
kernel_tensor = torch.Tensor(kernel).reshape(1, 1, -1)

for epoch in range(1000):
    optimizer.zero_grad()
    outputs = net(X_tensor)
    loss_value = torch.mean((outputs - y_tensor) ** 2)
    loss_value.backward()
    optimizer.step()

    running_loss += loss_value.item()

    if epoch % 100 == 0:
        print('[%d] loss: %.3f' % (epoch, loss_value.item()))
        print(net.weight.data.numpy())

prediction = (net(X_tensor).data).float()
prediction = (prediction.numpy().flatten())
data.Rainfall.plot()
plt.plot(prediction)
plt.title('Predicted Rainfall using conv1d')
plt.ylabel("Rainfall")
plt.show()


AttributeError: 'super' object has no attribute '_CNN1_Net__init'